In [0]:
%sql
-- lêr dados curados do Silver
CREATE OR REPLACE TEMP VIEW silver_fx AS
SELECT *
FROM delta.`abfss://silver@stgflowdev001.dfs.core.windows.net/fx_usdbrl/delta`;


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW gold_fx_stage AS
WITH base AS (
  SELECT
    ref_date,
    base,
    symbol,
    CAST(rate AS DOUBLE) AS rate,
    provider,
    ingestion_ts
  FROM silver_fx
),
win AS (
  SELECT
    *,
    LAG(rate) OVER (ORDER BY ref_date)                                         AS rate_prev,
    AVG(rate) OVER (ORDER BY ref_date ROWS BETWEEN 6  PRECEDING AND CURRENT ROW) AS ma7,
    AVG(rate) OVER (ORDER BY ref_date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS ma30
  FROM base
)
SELECT
  ref_date,
  base,
  symbol,
  rate,
  rate_prev,
  (rate - rate_prev) AS rate_d1_change,
  ma7,
  ma30,
  provider,
  ingestion_ts,
  CURRENT_TIMESTAMP() AS snapshot_ts,
  YEAR(ref_date)      AS year,
  MONTH(ref_date)     AS month
FROM win;


In [0]:
%sql
CREATE TABLE IF NOT EXISTS delta.`abfss://gold@stgflowdev001.dfs.core.windows.net/fx_usdbrl/metrics_delta`
(
  ref_date        DATE,
  base            STRING,
  symbol          STRING,
  rate            DOUBLE,
  rate_prev       DOUBLE,
  rate_d1_change  DOUBLE,
  ma7             DOUBLE,
  ma30            DOUBLE,
  provider        STRING,
  ingestion_ts    TIMESTAMP,
  snapshot_ts     TIMESTAMP,
  year            INT,
  month           INT
)
USING delta
PARTITIONED BY (year, month);


In [0]:
%sql
MERGE INTO delta.`abfss://gold@stgflowdev001.dfs.core.windows.net/fx_usdbrl/metrics_delta` AS t
USING gold_fx_stage AS s
ON  t.ref_date = s.ref_date
AND t.base     = s.base
AND t.symbol   = s.symbol
WHEN MATCHED THEN UPDATE SET
  t.rate           = s.rate,
  t.rate_prev      = s.rate_prev,
  t.rate_d1_change = s.rate_d1_change,
  t.ma7            = s.ma7,
  t.ma30           = s.ma30,
  t.provider       = s.provider,
  t.ingestion_ts   = s.ingestion_ts,
  t.snapshot_ts    = s.snapshot_ts,
  t.year           = s.year,
  t.month          = s.month
WHEN NOT MATCHED THEN INSERT *
;


In [0]:
%sql
SELECT ref_date, base, symbol, rate, rate_prev, rate_d1_change, ma7, ma30, provider
FROM delta.`abfss://gold@stgflowdev001.dfs.core.windows.net/fx_usdbrl/metrics_delta`
ORDER BY ref_date DESC
LIMIT 50;


Databricks visualization. Run in Databricks to view.